In [72]:
import pandas as pd

In [73]:
csv_1 = pd.read_csv('best_prompt.csv', index_col=0)[:4000]

In [74]:
csv_2 = pd.read_csv('best_prompt2.csv', index_col=0, sep='|')
csv_1 = pd.concat([csv_1, csv_2])
csv_1

,premise,hypothesis,rationales
0,A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Answer: Contradiction\nRationale: The hypot...
1,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Answer: Contradiction\nRationale: The hypot...
2,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Answer: Entailment\nRationale: The hypothes...
3,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Answer: Entailment\nRationale: The hypothes...
4,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Answer: Neutral\nRationale: The hypothesis ...
...,...,...,...
5587,Three women walk down a city street; one looks...,Three women chat at home.,8. Answer: Contradiction\nRationale: The premi...
5588,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. Answer: Neutral\nRationale: The premise des...
5589,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. Answer: Entailment\nRationale: The premise...
5590,Three naked little boys are playing in a river...,Three boys are playing in mud while one looks on.,Answer: Neutral Rationale: The hypothesis prov...


In [75]:
# from datasets import load_dataset
# import pandas as pd

# dataset = load_dataset("esnli")
# train_df = pd.DataFrame(dataset['train']).sample(frac=1/100, random_state=0)
# validation = pd.DataFrame(dataset['validation']).sample(frac=1/100, random_state=0)

# train_df = pd.concat([train_df, validation], ignore_index=True)

In [76]:
prompt_template= '''\
Premise: %s, 
Hypothesis: %s.
with options "entailment", "neutral", "contradiction"? Your answer must be less than 400 length, lexical diversity greater than 0.7 and structure is answer and rationales.
'''

In [77]:
error = csv_1[csv_1.rationales.isna()]

In [78]:
error['prompt'] = error.apply(lambda x: prompt_template % (x['premise'], x['hypothesis']), axis=1)

In [79]:
error

,premise,hypothesis,rationales,prompt


In [80]:
print(error.prompt[:10].values)

[]


In [82]:
csv_1['split'] = csv_1.rationales.apply(lambda x: x.lower().split('most likely answer'))

In [83]:
csv_1.split = csv_1.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [84]:
csv_1['correct_index'] = [{} for _ in range(len(csv_1))]
for index, row in csv_1.iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)


In [87]:
csv_1[csv_1.LLM_answer.isna()]

for index, row in csv_1[csv_1.LLM_answer.isna()].iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory', 'entail', 'contradict']:
        if choice in row['consensus_wucs_score'].lower():
            row.loc['correct_index'][choice] = row['consensus_wucs_score'].lower().index(choice)

In [88]:
import numpy as np
csv_1['LLM_answer'] =csv_1.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [89]:
csv_1[csv_1.LLM_answer.isna()]

,premise,hypothesis,rationales,split,correct_index,LLM_answer


In [481]:
# csv_1[csv_1.LLM_answer == 'entailed'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'entails'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'contradicts'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradicted'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradictory'] = 'contradiction'

In [90]:
paper = pd.read_csv('../paper.csv', index_col=0)

In [91]:
csv_1['question'] = csv_1['premise'] + '</s>' + csv_1['hypothesis']
csv_1.set_index('question', inplace=True)

In [92]:
paper[~paper.premise.isin(csv_1.premise)]['hypothesis'].values

array([], dtype=object)

In [93]:
paper.shape

(5592, 6)

In [94]:
total_df = csv_1[(csv_1.premise.isin(paper.premise)) & (csv_1.hypothesis.isin(paper.hypothesis))].drop_duplicates(subset=['premise', 'hypothesis'])
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Answer: Contradiction\nRationale: The hypot...,1. answer: contradiction\nrationale: the hypot...,"{'contradiction': 11, 'contradicts': 108}",contradiction
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Answer: Contradiction\nRationale: The hypot...,2. answer: contradiction\nrationale: the hypot...,"{'contradiction': 11, 'contradicts': 101}",contradiction
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Answer: Entailment\nRationale: The hypothes...,3. answer: entailment\nrationale: the hypothes...,{'entailment': 11},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Answer: Entailment\nRationale: The hypothes...,4. answer: entailment\nrationale: the hypothes...,{'entailment': 11},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Answer: Neutral\nRationale: The hypothesis ...,5. answer: neutral\nrationale: the hypothesis ...,"{'neutral': 11, 'contradicted': 130}",neutral
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,8. Answer: Contradiction\nRationale: The premi...,8. answer: contradiction\nrationale: the premi...,"{'contradiction': 11, 'contradicts': 216}",contradiction
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. Answer: Neutral\nRationale: The premise des...,9. answer: neutral\nrationale: the premise des...,{'neutral': 11},neutral
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. Answer: Entailment\nRationale: The premise...,10. answer: entailment\nrationale: the premise...,"{'entailment': 12, 'entails': 281}",entailment


In [95]:
total_df.loc[total_df.LLM_answer == 'entailed', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'entails', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradicts', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradicted', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradictory', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'entail', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradict', 'LLM_answer'] = 'contradiction'

In [96]:
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Answer: Contradiction\nRationale: The hypot...,1. answer: contradiction\nrationale: the hypot...,"{'contradiction': 11, 'contradicts': 108}",contradiction
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Answer: Contradiction\nRationale: The hypot...,2. answer: contradiction\nrationale: the hypot...,"{'contradiction': 11, 'contradicts': 101}",contradiction
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Answer: Entailment\nRationale: The hypothes...,3. answer: entailment\nrationale: the hypothes...,{'entailment': 11},entailment
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Answer: Entailment\nRationale: The hypothes...,4. answer: entailment\nrationale: the hypothes...,{'entailment': 11},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Answer: Neutral\nRationale: The hypothesis ...,5. answer: neutral\nrationale: the hypothesis ...,"{'neutral': 11, 'contradicted': 130}",neutral
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,8. Answer: Contradiction\nRationale: The premi...,8. answer: contradiction\nrationale: the premi...,"{'contradiction': 11, 'contradicts': 216}",contradiction
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. Answer: Neutral\nRationale: The premise des...,9. answer: neutral\nrationale: the premise des...,{'neutral': 11},neutral
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. Answer: Entailment\nRationale: The premise...,10. answer: entailment\nrationale: the premise...,"{'entailment': 12, 'entails': 281}",entailment


In [97]:
paper['my_label'] = total_df['LLM_answer']

In [98]:
sum(paper.label == paper.my_label) / len(paper)

0.623211731044349

In [99]:
sum(paper.label == paper.llm_label) / len(paper)

0.699928469241774

In [100]:
total_df.to_csv('best_prompt - full.csv')

In [195]:
csv_1.to_csv('neutral - full.csv', index=False)

In [101]:
total_df.index.value_counts()

A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.                                                                     1
a man taking a pizza out of an oven</s>A woman is sitting in the oven.                                                                                                                                                                  1
Side view of a mountaineer walking into strong headwinds.</s>A man outside.                                                                                                                                                             1
A boy jumping off of a diving board while a girl watches.</s>A boy is near water.                                                                                                                                                       1
A back view of a man driving a tractor and signaling with two ch